In [1]:
import pandas as pd
import tester
from trainer import read_tables
import os
import argparse
import json
from IPython.display import display, clear_output
import time


/home/cc/code/table_discovery_project/pyenv/s2ld/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_args(dataset):
    work_dir=os.path.abspath('..')
    test_args = argparse.Namespace(work_dir=work_dir,
                                   query_dir='demo_query',
                                   dataset=dataset)
    return test_args

In [3]:
def create_query_file(data_dir, question):
    file_name = 'fusion_query.jsonl'
    data_file = os.path.join(data_dir, file_name)
    query_data = {
        "id":0,
        "question":question,
        "table_id_lst":['N/A'],
        "answers":["N/A"],
        "ctxs": [{"title": "", "text": "This is a example passage."}]
    }
    with open(data_file, 'w') as f_o:
        f_o.write(json.dumps(query_data) + '\n')
    

In [4]:
def show_results(question, out_dir, table_dict):
    data_file = os.path.join(out_dir, 'pred_epoch_0_None.jsonl')
    with open(data_file) as f:
        item = json.load(f)
    tag_lst = item['tags']
    N = 5
    pred_table_set = set()
    for tag in tag_lst:
        table_id = tag['table_id']
        if table_id not in pred_table_set:
            pred_table_set.add(table_id)
            table = table_dict[table_id]
            show_table(question, table, len(pred_table_set))
        if len(pred_table_set) >= 5:
            return
    
def show_table(question, table, seq_no):
    print("Question, %s" % question)
    print("Top %d table,         %s " % (seq_no, table['documentTitle']))
    col_lst = table['columns']
    col_names = [a['text'] for a in col_lst]
    row_lst = table['rows']
    row_data = []
    for row_item in row_lst:
        cell_lst = row_item['cells']
        row_item = [a['text'] for a in cell_lst]
        row_data.append(row_item)
    df = pd.DataFrame(row_data,columns=col_names)
    display(df)
    

In [7]:
dataset=input('Type the dataset, ')
args = get_args(dataset)
print('Loading tables')
table_dict = read_tables(args.work_dir, dataset)
print('Loading index')
args.table_repre = 'rel_graph'
index_obj = tester.get_index_obj(args.work_dir, dataset, args)
data_dir = os.path.join(args.work_dir, 'data', dataset, 'demo_query', 'test')
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)
question = input('Type a question (q to quit),\n').strip()
args.train_model_dir = None
args.bnn = 1
while question != 'q':
    create_query_file(data_dir, question)
    out_dir = tester.main(args, table_data=table_dict, index_obj=index_obj)
    clear_output(wait=False)
    show_results(question, out_dir, table_dict)
    time.sleep(1)
    question = input('Type a question (q to quit),\n').strip()
    

FileNotFoundError: [Errno 2] No such file or directory: '/home/cc/code/table_discovery_project/open_table_discovery/output/fetaqa/test_ccdcef11-d7b8-4d51-8195-ea00b05c8030/pred_None.jsonl'

Example questions in fetaqa dataset,

1. What shows did Marisol Nichols appear in during 1996?

2. What and when were Tom Felton's first two roles?

3. How close was Texas' 21st congressional district election?